In [10]:
# using bronzebeard

from subprocess import Popen, PIPE
from sys import argv
from binascii import hexlify


#fname = argv[1]
fname = "fib.S"
fname_woext = fname.split(".")[0]
fname_hex = f"{fname_woext}.hex"
fname_v =  f"{fname_woext}.vv"

cmd = Popen(["bronzebeard", fname, "-o", fname_hex], stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)

_, err = cmd.communicate(b"")

code_hex = []

with open (fname_hex, "rb") as f:
    code_hex = f.read()

code = []

word_size = 4
for i in range(0, len(hexa), word_size):
    word = code_hex[i:i+word_size]
    word = word[::-1]
    code.append(hexlify(word).decode())


with open(fname_v, "w+") as f:
    f.write("task LOADMEM;\n")
    f.write("\tbegin\n")
    for i, line in enumerate(code):
        f.write(f"\t\tMEM[{i}] = 32'h{line};\n")
    f.write("\tend\n")
    f.write("endtask\n")

['00100313', '00100393', '00000e13', '00030513', '00730e33', '00030393', '000e0313', '00030513', 'ff1ff06f']


In [9]:
from subprocess import Popen, PIPE

fname = "fib.S"
fname_woext = fname.split(".")[0]
fname_elf = f"{fname_woext}.elf"
fname_o = f"{fname_woext}.o"
fname_v =  f"{fname_woext}.vv"

asm = ["riscv64-unknown-elf-as", "-march=rv32i", "-mabi=ilp32", "-mno-relax", fname, "-o", fname_o]
link = ["riscv64-unknown-elf-ld", fname_o, "-o", fname_elf, "-T", "tang.ld", "-m", "elf32lriscv", "-nostdlib", "-no-relax"]

cmd_asm = Popen(asm, stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)
_, err = cmd_asm.communicate(b"")

if (err): print(err)

cmd_link = Popen(link, stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)
output, err = cmd_asm.communicate(b"")

if (err): print(err)
if (output): print(output)

# riscv64-unknown-elf-ld blinker.o wait.o -o blinker.bram.elf -T bram.ld -m elf32lriscv -nostdlib -no-relax


In [33]:
# using gcc

from subprocess import Popen, PIPE
from sys import argv

fname = argv[1]
fname_woext = fname.split(".")[0]


cmd_as = Popen(["riscv64-unknown-elf-as", "-march=rv32i", "-mabi=ilp32", "-mno-relax", fname, "-o", f"{fname_woext}.elf"], stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)

_, err = cmd_as.communicate(b"")

cmd_objdump = Popen(["riscv64-unknown-elf-objdump.exe", "-S", f"{fname_woext}.elf"], stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)

code, err = cmd_objdump.communicate(b"")

code = code.decode("utf-8")

codet = [x.strip() for x in code.split("\n") if "\t" in x] # extract lines containing \t
code_hex = [(x.split("\t")[1]).strip() for x in codet] # extract hex instructions


with open(f"{fname_woext}.hex", "w+") as f:
    f.write("v2.0 raw\n")
    for line in code_hex:
        f.write(f"{line}\n")


